In [68]:
import pandas as pd
import numpy as np
bookings = pd.read_csv('bookings.csv', sep = ';')

In [71]:
## информация об исходном датафрейме
bookings.shape
bookings.info()
#bookings_head = bookings.head(7)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 119390 entries, 0 to 119389
Data columns (total 21 columns):
 #   Column                     Non-Null Count   Dtype  
---  ------                     --------------   -----  
 0   Hotel                      119390 non-null  object 
 1   Is Canceled                119390 non-null  int64  
 2   Lead Time                  119390 non-null  int64  
 3   arrival full date          119390 non-null  object 
 4   Arrival Date Year          119390 non-null  int64  
 5   Arrival Date Month         119390 non-null  object 
 6   Arrival Date Week Number   119390 non-null  int64  
 7   Arrival Date Day of Month  119390 non-null  int64  
 8   Stays in Weekend nights    119390 non-null  int64  
 9   Stays in week nights       119390 non-null  int64  
 10  stays total nights         119390 non-null  int64  
 11  Adults                     119390 non-null  int64  
 12  Children                   119386 non-null  float64
 13  Babies                     11

In [73]:
## функция, переименовывающая название столбца
def modify_column(name):
    new_name = name.replace(' ', '_').lower()
    return new_name
## переименовывание столбцов
bookings = bookings.rename(columns = modify_column)

In [74]:
## число успешных бронирований по каждой стране
bookings.query('is_canceled == 0').country.value_counts()

PRT    21071
GBR     9676
FRA     8481
ESP     6391
DEU     6069
       ...  
BHR        1
DJI        1
MLI        1
NPL        1
FRO        1
Name: country, Length: 165, dtype: int64

In [75]:
## среднее число ночей в отеле City Hotel = 2.98
mean_city_hotel = round(bookings.query("hotel == 'City Hotel'").stays_total_nights.mean(), 2)
## среднее число ночей в отеле Resort Hotel = 4.32
mean_resort_hotel = round(bookings.query("hotel == 'Resort Hotel'").stays_total_nights.mean(), 2)
print(mean_resort_hotel, mean_city_hotel)

4.32 2.98


In [76]:
## число случаев овербукинга: когда полученный тип номера отличается от забронированного
bookings.query('assigned_room_type != reserved_room_type').shape

(14917, 21)

In [77]:
## самое большое число бронирований в 2016 году приходится на октябрь
book_2016 = bookings.query('arrival_date_year == 2016')
book_2016.groupby(by = 'arrival_date_month', as_index = False).agg({'is_canceled' : 'count'}).sort_values(
    by = 'is_canceled', ascending = False)

## самое большое число бронирований в 2017 году приходится на май
book_2017 = bookings.query('arrival_date_year == 2017')
book_2017.groupby(by = 'arrival_date_month', as_index = False).agg({'is_canceled' : 'count'}).sort_values(
    by = 'is_canceled', ascending = False)

,arrival_date_month,is_canceled
7,May,6313
0,April,5661
5,June,5647
4,July,5313
6,March,4970
1,August,4925
2,February,4177
3,January,3681


In [78]:
## в каком месяце чаще всего отменяли бронь City Hotel
hotel = bookings.query("hotel == 'City Hotel' and is_canceled == 1")
hh = hotel.groupby(by = 'arrival_date_year', as_index = False)['arrival_date_month'].value_counts()
hh.sort_values(by = 'count', ascending = False)

,arrival_date_year,arrival_date_month,count
18,2017,May,2217
6,2016,October,1947
19,2017,April,1926
20,2017,June,1808
7,2016,June,1720
8,2016,September,1567
0,2015,September,1543
9,2016,April,1539
10,2016,May,1436
11,2016,November,1360


In [79]:
## среднее число взрослых, малышей и детей
print(bookings.adults.mean(), bookings.babies.mean(), bookings.children.mean())

1.8564033838679956 0.007948739425412514 0.10388990333874994


In [80]:
## создание общего столбца, учитывающего количество детей и подсчет их среднего числа для разного типа отелей
bookings['total_kids'] = bookings['babies'] + bookings['children']
print(round(bookings.query("hotel == 'City Hotel'").total_kids.mean(), 2))
print(round(bookings.query("hotel == 'Resort Hotel'").total_kids.mean(), 2))

0.1
0.14


In [98]:
## создание нового логического столбца - есть детей / нет детей
## вычисление метрики churn rate - оттока клиентов, в зависимости от has_kids 
bookings['has_kids'] = np.where(bookings['total_kids'] >= 1, True, False)
## имеют детей - 9332
bookings.query('has_kids == True').shape
## имеют детей и отменили бронь - 3259. 35.0% 
bookings.query('has_kids == True and is_canceled == 1').shape
## не имеют детей - 110058
bookings.query('has_kids == False').shape
## не имеют детей и отменили бронь - 40965. 37.0%
bookings.query('has_kids == False and is_canceled == 1').shape

round(3259 / 9332, 2)
round(40965 / 110058, 2)

0.37